# mbGDML Model and Data Set UMAP

Any cells that could require manual change is marked with the comment `# CHANGE` at the top.

In [1]:
import os
import numpy as np

from mbgdml import utils
from mbgdml.data import mbModel
from mbgdml.data import dataSet
from mbgdml.predict import mbPredict
from mbgdml.analysis.structures import structureEmbedding

from ase import Atoms
from ase.visualize import view
import nglview

Information and data needs to be collected from a variety of sources and is curated.
Instead of recurating and recalculating everything for a reproducible UMAP, we provide a way to save a load in a numpy npz file.
The next cell selects whether we are going to `'calculate'` or `'load'` in our data.

In [2]:
# CHANGE
mode = 'calculate'  # 'calculate' or 'load'

Next we need to specify where we are getting our data to calculate or load.

## Calculate

If we are going to calculate a new UMAP, we need to list paths to the npz file and which type of mbGDML data it is.
We also need to specify a `save_dir` and `umap_name` to save this UMAP to.
The three paths we need to specify are

- `model_path`: Path to a trained mbGDML model.
- `model_dset_path`: Path to the many-body data set that the model was trained on (after the < n body contributions were removed). This is used to determine the true/reference energies and forces.
- `dset_path`: Path to the data set to be included in the UMAP embedding (with all n-body contributions).

We also need to specify paths to create a temporary many-body data set for `dset_path` in the variable `mb_model_paths`.

## Load

Only the path to the saved UMAP data is needed.


In [3]:
# CHANGE
overwrite = True

if mode == 'calculate':
    # The model to analyze.
    model_path = '../../data/models/h2o/3h2o/140h2o.sphere.gfn2.md.500k.prod1.3h2o.cm10-model.mb-iterativetrain1000.npz'
    # The data set used for training.
    model_dset_path = '../../data/datasets/h2o/3h2o/140h2o.sphere.gfn2.md.500k.prod1.3h2o-dset.mb-cm10.npz'
    # The data set we want to analyze against a model.
    dset_path = '../../data/datasets/h2o/3h2o/16h2o.yoo.etal.boat.b.3h2o-dset.mb-cm10.npz'
    # If you want to remove n-body contributions from the dset, these paths are for the lower order models.
    mb_model_paths = [

    ]
    
    save_dir = '../../analysis/umaps'
    umap_name = '16h2o.yoo.etal.boat.b.3h2o.cm10-umap.mb-140h2o.sphere.gfn2.md.500k.prod1.3h2o.cm10.iterativetrain1000'

    # Do not need to change anything below this line for 'calculate'.
    if save_dir[-1] != '/':
        save_dir += '/'
    umap_path = save_dir + umap_name + '.npz'

    if os.path.isfile(umap_path) and not overwrite:
        raise FileExistsError(f'{umap_path} already exists and overwrite is False')

    # Tries to avoid recalculating embedding.
    try:
        if _calculated == True:
            pass
    except NameError:
        _calculated = False

elif mode == 'load':
    umap_path = '../../analysis/umaps/umap-112h2o.box.pm.gfn2.md.3h2o.model-12h2o.su.etal.3h2o.dset.npz'

else:
    raise ValueError(f'{mode} is not a valid option.')



These are just some useful utility functions.

In [4]:
def get_filename(path):
    """The name of the file without the extension from a path.

    If there are periods in the file name with no file extension, will
    always remove the last one.

    Parameters
    ----------
    path : :obj:`str`
        Path to file.

    Returns
    -------
    :obj:`str`
        The file name without an extension.
    """
    return os.path.splitext(os.path.basename(path))[0]

In [5]:
def umap_embed(umap_data, n_neighbors, min_dist):
    """Performs the embedding 

    Parameters
    ----------
    umap_data : :obj:`mbgdml.analysis.structureEmbedding`
        An already initialized and calculated UMAP data object.
    n_neighbors : :obj:`int`
        The size of the local neighborhood. Smaller numbers will push data
        into their own regions whereas larger numbers promote a more
        uniform, global picture.
    min_dist : :obj:`float`
        The minimum distance between two points. Lower values enable UMAP
        to place points closer to each other. For these descriptors, lower
        values are generally recommended to represent similar structures.
    """
    umap_data.n_neighbors = n_neighbors
    umap_data.min_dist = min_dist

    umap_data.embed()
    return umap_data

## Main Routine for 'calculate'

The two main parameters for UMAP are `n_neighbors` and `min_dist`.
For more information on these parameters see [this website](https://umap-learn.readthedocs.io/en/latest/parameters.html).
Every UMAP for this purpose needs to have a `random_state` defined.
If one is not choosen (by setting `random_state = None`), a random number is choosen by default.

In [6]:
# CHANGE
n_neighbors = 10  # 5
min_dist = 0.1  # 0.1
random_state = None

In [7]:
def main_calculate(
    model_path, model_dset_path, dset_path, mb_model_paths,
    n_neighbors, min_dist, random_state=None
):
    """

    Parameters
    ----------
    model_path : :obj:`str`

    model_dset_path : :obj:`str`

    dset_path : :obj:`str`

    mb_model_paths : :obj:`list` [:obj:`str`]

    n_neighbors : :obj:`int`
        The size of the local neighborhood. Smaller numbers will push data
        into their own regions whereas larger numbers promote a more
        uniform, global picture.
    min_dist : :obj:`float`
        The minimum distance between two points. Lower values enable UMAP
        to place points closer to each other. For these descriptors, lower
        values are generally recommended to represent similar structures.
    random_state : :obj:`int` or :obj:`None`, optional
    """
    # INITIALIZES UMAP ANALYSIS OBJECT
    umap_data = structureEmbedding(
        n_neighbors=n_neighbors, min_dist=min_dist,
        random_state=random_state
    )

    # MODEL INFORMATION
    # Loading model and model data set.
    print('Gathering model data')
    model_dset = dataSet(model_dset_path)
    model = mbModel()
    model.load(model_path)
    train_idxs = model.model['idxs_train']
    model_info = {
        'name': get_filename(model_path),
        'md5': model.model['md5'],
        'type': 'm',
        'quantity': len(train_idxs)
    }

    # Getting UMAP relevant data for model contribution.
    umap_data.z = model.model['z']
    umap_data.R = model_dset.R[train_idxs]
    umap_data.R_desc = model.model['R_desc'].T

    umap_data.E_true = model_dset.E[train_idxs]
    umap_data.F_true = model_dset.F[train_idxs]

    # DATA SET INFORMATION
    # Loading data set.
    print('Gathering data set data')
    dset = dataSet(dset_path)
    dset_info = {
        'name': get_filename(dset_path),
        'md5': dset.md5,
        'type': 'd',
        'quantity': dset.n_R,
        'entity_ids': dset.entity_ids,
        'comp_ids': dset.comp_ids,
    }
    assert np.all((dset.z, umap_data.z))

    # Getting mb data set (removes < n contributions).
    if len(mb_model_paths) > 0:
        print('Creating many-body data set')
        dset_mb = dataSet()
        dset_mb.create_mb(dset, mb_model_paths)
        dset_mb_R_desc, _ = umap_data.get_R_desc(dset_mb.z, dset_mb.R)

        dset_R = dset_mb.R
        dset_R_desc = dset_mb_R_desc
        dset_E = dset_mb.E
        dset_F = dset_mb.F
    else:
        # Cannot make many-body data set with 1mer models.
        dset_R = dset.R
        dset_R_desc, _ = umap_data.get_R_desc(dset.z, dset.R)
        dset_E = dset.E
        dset_F = dset.F

    # Adding data set information.
    umap_data.R = np.concatenate((umap_data.R, dset_R), axis=0)
    umap_data.R_desc = np.concatenate(
        (umap_data.R_desc, dset_R_desc), axis=0
    )
    umap_data.E_true = np.concatenate((umap_data.E_true, dset_E), axis=0)
    umap_data.F_true = np.concatenate((umap_data.F_true, dset_F), axis=0)

    # DATA/SOURCE INFORMATION
    umap_data.data_info = [model_info, dset_info]

    # PREDICTION DATA
    print('Predicting energies and forces')
    predict = mbPredict([model_path])  # Needs to be a list.
    umap_data.E_pred = np.full(umap_data.E_true.shape, np.nan)
    umap_data.F_pred = np.full(umap_data.F_true.shape, np.nan)

    # Loop through all R and predict the energies and forces.
    for i in range(len(umap_data.R)):
        e, f = predict.predict(
            umap_data.z, umap_data.R[i],
            dset_info['entity_ids'], dset_info['comp_ids']
        )
        umap_data.E_pred[i] = e
        umap_data.F_pred[i] = f
    
    # EMBEDDING
    print('Calculating UMAP embedding')
    umap_data = umap_embed(
        umap_data, umap_data.n_neighbors, umap_data.min_dist
    )

    # RETURNING
    print('Done')
    return umap_data

In [8]:
if mode == 'calculate' and not _calculated:
    umap_data = main_calculate(
        model_path, model_dset_path, dset_path, mb_model_paths,
        n_neighbors, min_dist, random_state=random_state
    )
    umap_data.save(umap_name, umap_data.umap_data, save_dir)
    _calculated = True

Gathering model data
Gathering data set data
Predicting energies and forces
Calculating UMAP embedding
Done


## Main routine for 'load'

In [9]:
def main_load(umap_path):
    """

    Parameters
    ----------
    umap_path : :obj:`str`
        Path to UMAP data npz file.
    """
    umap_data = structureEmbedding()
    umap_data.load(umap_path)
    return umap_data

In [10]:
if mode == 'load':
    umap_data = main_load(umap_path)

## Parameter turning

In [11]:
import plotly.graph_objects as go

In [24]:
# Embedding parameters
n_neighbors = 5
min_dist = 0.05
umap_data = umap_embed(umap_data, n_neighbors, min_dist)

# umap_data.save(umap_name, umap_data.umap_data, save_dir)  # Uncomment if you want to save

# X,Y embedding coordinates
embedding_x = umap_data.embedding[:, 0]
embedding_y = umap_data.embedding[:, 1]

# COLORS
# Model is alway first, then the data set.
assert umap_data.data_info[0]['type'] == 'm'
assert umap_data.data_info[1]['type'] == 'd'

model_n_R = umap_data.data_info[0]['quantity']
dset_n_R = umap_data.data_info[1]['quantity']

# True Energy
model_E_true = umap_data.E_true[:model_n_R]
dset_E_true = umap_data.E_true[model_n_R:]

# Predicted Energy
model_E_pred = umap_data.E_pred[:model_n_R]
dset_E_pred = umap_data.E_pred[model_n_R:]

# Energy prediction error
E_error = umap_data.E_true - umap_data.E_pred
model_E_error = E_error[:model_n_R]
dset_E_error = E_error[model_n_R:]

# FIGURE 

fig = go.Figure()

smallest_size = 8  # 8
exp_scale = 20  # 1.3



model_customdata = np.dstack((model_E_true, model_E_error))[0]
fig.add_scatter(
    name='Model',
    x=embedding_x[0:model_n_R],
    y=embedding_y[0:model_n_R],
    mode='markers',
    marker_symbol='circle',
    marker=dict(
        color='#636EFA',
        size=smallest_size + np.exp(exp_scale*model_E_error),
        opacity=0.3,
        line=dict(
            width=1.5
        )
    ),
    customdata=model_customdata,
    hovertemplate="<br>".join([
        "%{text}",
        "E true: %{customdata[0]:.5f}",
        "E error: %{customdata[1]:.5f}"
    ]),
    text = [f'R index: {i}' for i in range(0, model_n_R)]
)

dset_customdata = np.dstack((dset_E_true, dset_E_error))[0]
fig.add_scatter(
    name='Data Set',
    x=embedding_x[model_n_R:],
    y=embedding_y[model_n_R:],
    mode='markers',
    marker_symbol='circle-open',
    marker=dict(
        color='#EF553B',
        size=smallest_size + np.exp(exp_scale*dset_E_error),
        opacity=0.7,
        line=dict(
            width=1.5
        )
    ),
    customdata=dset_customdata,
    hovertemplate="<br>".join([
        "%{text}",
        "E true: %{customdata[0]:.5f}",
        "E error: %{customdata[1]:.5f}"
    ]),
    text = [f'R index: {i}' for i in range(model_n_R, model_n_R + dset_n_R)]
)

# Background colors
fig.update_layout(
    width=1200,
    height=800,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(255, 255, 255 , 1)'
)


fig.show()

In [25]:
R_index = 1283
r = umap_data.R[R_index]
r_atoms = Atoms(numbers=umap_data.z, positions=r)
view(r_atoms, viewer='ngl')

## Playground

Cells below this can be changed to whatever.

In [26]:
R_index = 359
r = umap_data.R[R_index]
r_atoms = Atoms(numbers=umap_data.z, positions=r)
view(r_atoms, viewer='ngl')